In [59]:
import requests
import pandas as pd

NETWORK_URL = "https://api.citybik.es/v2/networks/bikesampa"

response = requests.get(NETWORK_URL)

if response.status_code == 200:
    data = response.json()
    stations = data["network"]["stations"]
    df_stations = pd.DataFrame(stations)
    display(df_stations.head(10))  # Mostra as 10 primeiras estações
else:
    print("Erro na requisição:", response.status_code)

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra
0,0495af0c779f79ebaa01242df456963d,204 - Consolação,-23.556727,-46.664197,2025-08-17T20:00:25.505310+00:00Z,0,19,"{'uid': '204', 'renting': 1, 'returning': 1, '..."
1,05ac42b0b09d9dda95118ee7ff00d5bc,150 - Metrô Paraíso,-23.575386,-46.641375,2025-08-17T20:00:25.504968+00:00Z,0,19,"{'uid': '150', 'renting': 1, 'returning': 1, '..."
2,07e1bc56a4d45c04055860ae39f9322a,7 - Praça Joaquim Roberto,-23.572544,-46.693681,2025-08-17T20:00:25.503928+00:00Z,10,9,"{'uid': '7', 'renting': 1, 'returning': 1, 'la..."
3,086f46ad981708395ea90ca6ae362832,434 - Praça Procópio Ferreira II,-23.599550,-46.690740,2025-08-17T20:00:25.506774+00:00Z,3,12,"{'uid': '592', 'renting': 1, 'returning': 1, '..."
4,0b509d7429586cacb0776fd8b203d61c,443 - Cardoso Melo III,-23.596440,-46.688550,2025-08-17T20:00:25.506569+00:00Z,0,15,"{'uid': '556', 'renting': 1, 'returning': 1, '..."
5,0c03fdb27d57954593e6da8f953d697f,14 - Museu da Casa Brasileira,-23.580880,-46.684709,2025-08-17T20:00:25.503999+00:00Z,8,9,"{'uid': '14', 'renting': 1, 'returning': 1, 'l..."
6,0d2a3ac5d13a76ec7974e997b3717b0b,519 - Praça Afrodisio Vidigal,-23.577480,-46.640810,2025-08-17T20:00:25.506697+00:00Z,0,14,"{'uid': '573', 'renting': 1, 'returning': 1, '..."
7,0d4298dc15738906a9ac9f3a70dc21cb,266 - Méqui Verde,-23.573640,-46.642140,2025-08-17T20:00:25.506850+00:00Z,0,7,"{'uid': '601', 'renting': 1, 'returning': 1, '..."
8,0e40d986af22d73ed98818286954e46d,517 - Travessa Hélio Pellegrino,-23.598740,-46.671440,2025-08-17T20:00:25.506950+00:00Z,2,13,"{'uid': '628', 'renting': 1, 'returning': 1, '..."
9,0ecb986a30ea8a70c99bd89d758a1e86,290 - Metrô Marechal Deodoro,-23.533742,-46.657056,2025-08-17T20:00:25.506281+00:00Z,18,1,"{'uid': '446', 'renting': 1, 'returning': 1, '..."


In [60]:
df_new = pd.DataFrame(data['network']['stations'])

In [62]:
df_both = df_old.merge(df_new[['id','timestamp','free_bikes','empty_slots']], on = 'id', how = 'left', suffixes = ('_old','_new'))

In [72]:
key_list = []
for station in data['network']['stations']:
    key_list = key_list + list(data['network']['stations'][0]['extra'].keys())

In [73]:
key_list = list(set(key_list))

In [75]:
extra_dict = []
for station in data['network']['stations']:
    for key in data['network']['stations'][0]['extra'].keys():
        if key not in key_list:
            print(f"Station {station['name']} does not have {key}")

In [98]:
def clean_stations(data):
    stations = []
    for station in data['network']['stations']:
        station_data = {}
        for key in station.keys():
            if key != 'extra':
                station_data[key] = station[key]
            else:
                for extra_key in station['extra'].keys():
                    station_data[extra_key] = station['extra'][extra_key]
        stations.append(station_data)
    
    return pd.DataFrame(stations)

In [99]:
def get_distances(data):
    supersim_lat = -23.556641
    supersim_lng = -46.681632
    distances = []
    for station in data['network']['stations']:
        lat = station['latitude']
        lng = station['longitude']
        p1 = (lat, lng)
        p2 = (supersim_lat, supersim_lng)
        distance = geodesic(p1, p2).km
        distances.append(distance)
    return distances

In [100]:
df_stations = clean_stations(data)

In [101]:
df_stations

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,uid,renting,returning,...,post_code,has_ebikes,ebikes,normal_bikes,payment,payment-terminal,altitude,slots,rental_uris,virtual
0,0495af0c779f79ebaa01242df456963d,204 - Consolação,-23.556727,-46.664197,2025-08-17T20:00:25.505310+00:00Z,0,19,204,1,1,...,03164200,True,0,0,"[key, transitcard, phone]",False,-23.56,19,{},False
1,05ac42b0b09d9dda95118ee7ff00d5bc,150 - Metrô Paraíso,-23.575386,-46.641375,2025-08-17T20:00:25.504968+00:00Z,0,19,150,1,1,...,NaN,True,0,0,"[key, transitcard, creditcard, phone]",True,NaN,19,{},False
2,07e1bc56a4d45c04055860ae39f9322a,7 - Praça Joaquim Roberto,-23.572544,-46.693681,2025-08-17T20:00:25.503928+00:00Z,10,9,7,1,1,...,03164200,True,0,10,"[key, transitcard, phone]",False,NaN,19,{},False
3,086f46ad981708395ea90ca6ae362832,434 - Praça Procópio Ferreira II,-23.599550,-46.690740,2025-08-17T20:00:25.506774+00:00Z,3,12,592,1,1,...,04533-085,True,0,3,"[key, transitcard, phone]",False,0.00,15,{},False
4,0b509d7429586cacb0776fd8b203d61c,443 - Cardoso Melo III,-23.596440,-46.688550,2025-08-17T20:00:25.506569+00:00Z,0,15,556,1,1,...,04548-005,True,0,0,"[key, transitcard, phone]",False,0.00,15,{},False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,fd5d8d6dcc34499ffa8ca4db591e8e45,317 - R. Leonardo Nunes,-23.598733,-46.639103,2025-08-17T20:00:25.505845+00:00Z,1,14,317,1,1,...,03164200,True,0,1,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False
236,fdb890213c54ad952730bd9fab78a147,76 - 8° Batalhão,-23.574298,-46.654040,2025-08-17T20:00:25.504419+00:00Z,2,13,76,1,1,...,03164200,True,0,2,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False
237,fe23e24943b116bf2b7bbfe856292ffa,131 - Ibirapuera Portão 06,-23.593685,-46.660042,2025-08-17T20:00:25.504800+00:00Z,13,6,131,1,1,...,03164200,True,0,13,"[key, transitcard, phone]",False,NaN,19,{},False
238,ff724047677a832632b2ec87805270c0,215 - E.E. Godofredo Furtado,-23.559442,-46.678456,2025-08-17T20:00:25.505348+00:00Z,1,14,215,1,1,...,NaN,True,0,1,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False


In [102]:
len(get_distances(data))

240

In [103]:
df_stations['distance_to_supersim'] = get_distances(data)

In [106]:
df_stations.sort_values(by = 'distance_to_supersim', ascending = True)

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,uid,renting,returning,...,has_ebikes,ebikes,normal_bikes,payment,payment-terminal,altitude,slots,rental_uris,virtual,distance_to_supersim
163,af172dc3045f2f2730d3959b34d17b75,56 - Henrique Schaumann,-23.557886,-46.681845,2025-08-17T20:00:25.506318+00:00Z,2,9,458,1,1,...,True,0,2,"[key, transitcard, phone]",False,0.0,11,{},False,0.139616
238,ff724047677a832632b2ec87805270c0,215 - E.E. Godofredo Furtado,-23.559442,-46.678456,2025-08-17T20:00:25.505348+00:00Z,1,14,215,1,1,...,True,0,1,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False,0.448757
43,30281ec20f9f0d4ba1c5a12021a92f3b,222 - Rua Horácio Lane,-23.558739,-46.687131,2025-08-17T20:00:25.505398+00:00Z,0,15,222,1,1,...,True,0,0,"[key, transitcard, phone]",False,NaN,15,{},False,0.607654
179,baa6c2785d8b8bc50181c7ee5051c3db,219 - R. Joaquim Antunes,-23.562392,-46.682927,2025-08-17T20:00:25.505372+00:00Z,3,12,219,1,1,...,True,0,3,"[key, transitcard, phone]",False,NaN,15,{},False,0.650490
42,2f99b00ceacd95133db79cdaa22072a8,216 - João Moura 328,-23.562142,-46.676536,2025-08-17T20:00:25.505360+00:00Z,4,11,216,1,1,...,True,0,4,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False,0.801168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,b856bb9abec5cc4da18b06f51a159d7b,477 - Rua Arizona,-23.614898,-46.687426,2025-08-17T20:00:25.505665+00:00Z,7,12,280,1,1,...,True,0,7,"[key, transitcard, phone]",False,NaN,19,{},False,6.479130
198,d4ac3e3a43612ad25c40e475f06f5e1a,333 - Rua Flórida 53,-23.617014,-46.681398,2025-08-17T20:00:25.505909+00:00Z,8,7,333,1,1,...,True,0,8,"[key, transitcard, phone]",False,NaN,15,{},False,6.686471
122,839b5b7c69c403d75fbdd0a99f5f685d,339 - R. Min. Luiz Gallotti,-23.618814,-46.683919,2025-08-17T20:00:25.505959+00:00Z,3,24,339,1,1,...,True,0,3,"[key, transitcard, phone]",False,NaN,27,{},False,6.889802
89,6197cbb5cb8c87be28262b1df4564f19,311 - Luis Góis,-23.604226,-46.636996,2025-08-17T20:00:25.506382+00:00Z,3,12,469,1,1,...,True,0,3,"[key, transitcard, creditcard, phone]",True,0.0,15,{},False,6.966754


In [89]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1920 non-null   object 
 1   name              1920 non-null   object 
 2   latitude          1920 non-null   float64
 3   longitude         1920 non-null   float64
 4   timestamp         1920 non-null   object 
 5   free_bikes        1920 non-null   int64  
 6   empty_slots       1920 non-null   int64  
 7   uid               1920 non-null   object 
 8   renting           1920 non-null   int64  
 9   returning         1920 non-null   int64  
 10  last_updated      1912 non-null   float64
 11  address           1912 non-null   object 
 12  post_code         1640 non-null   object 
 13  has_ebikes        1920 non-null   bool   
 14  ebikes            1920 non-null   int64  
 15  normal_bikes      1920 non-null   int64  
 16  payment           1912 non-null   object 


In [107]:
df_stations.columns

Index(['id', 'name', 'latitude', 'longitude', 'timestamp', 'free_bikes',
       'empty_slots', 'uid', 'renting', 'returning', 'last_updated', 'address',
       'post_code', 'has_ebikes', 'ebikes', 'normal_bikes', 'payment',
       'payment-terminal', 'altitude', 'slots', 'rental_uris', 'virtual',
       'distance_to_supersim'],
      dtype='object')

In [ ]:
df_stations

In [113]:
', '.join(df_stations[['id','timestamp','name','free_bikes','empty_slots','slots','distance_to_supersim','renting','returning','has_ebikes','ebikes','normal_bikes','payment','payment-terminal']].columns)

'id, timestamp, name, free_bikes, empty_slots, slots, distance_to_supersim, renting, returning, has_ebikes, ebikes, normal_bikes, payment, payment-terminal'

In [118]:
import psycopg2

In [116]:
!pip install psycopg2

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 20.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [121]:
conn = psycopg2.connect("dbname=weather_db user=postgres password=postgres host=localhost")
cur = conn.cursor()

for s in df_stations:
    cur.execute("""
        INSERT INTO bike_stations (id, updated_at, station_name, free_bikes, empty_slots, slots, distance_to_supersim, renting, returning, has_ebikes, ebikes, normal_bikes, payment, payment_terminal)
        VALUES (%s, NOW(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (station_id) DO NOTHING
    """, (s["id"], s["name"], s["free_bikes"], s["empty_slots"], s["slots"], s["distance_to_supersim"], s["renting"], s["returning"], s["has_ebikes"], s["ebikes"], s["normal_bikes"], s["payment"], s["payment_terminal"]))

conn.commit()
cur.close()
conn.close()

TypeError: string indices must be integers, not 'str'

In [136]:
conn = psycopg2.connect(
    dbname="weather_db",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Insert row by row
insert_query = """
    INSERT INTO public.citybik_log (id, updated_at, station_name, free_bikes, empty_slots, address, distance_to_supersim, renting, "returning", has_ebikes, ebikes, normal_bikes, payment_terminal)
    VALUES (%s, NOW(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT DO NOTHING;
"""

for _, row in df_stations[['id','name','free_bikes','empty_slots','address','distance_to_supersim','renting','returning','has_ebikes','ebikes','normal_bikes','payment-terminal']].iterrows():
    cur.execute(insert_query, tuple(row))

conn.commit()
cur.close()
conn.close()

In [135]:
df_stations

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,uid,renting,returning,...,has_ebikes,ebikes,normal_bikes,payment,payment-terminal,altitude,slots,rental_uris,virtual,distance_to_supersim
0,0495af0c779f79ebaa01242df456963d,204 - Consolação,-23.556727,-46.664197,2025-08-17T20:00:25.505310+00:00Z,0,19,204,1,1,...,True,0,0,"[key, transitcard, phone]",False,-23.56,19,{},False,1.780092
1,05ac42b0b09d9dda95118ee7ff00d5bc,150 - Metrô Paraíso,-23.575386,-46.641375,2025-08-17T20:00:25.504968+00:00Z,0,19,150,1,1,...,True,0,0,"[key, transitcard, creditcard, phone]",True,NaN,19,{},False,4.604427
2,07e1bc56a4d45c04055860ae39f9322a,7 - Praça Joaquim Roberto,-23.572544,-46.693681,2025-08-17T20:00:25.503928+00:00Z,10,9,7,1,1,...,True,0,10,"[key, transitcard, phone]",False,NaN,19,{},False,2.148335
3,086f46ad981708395ea90ca6ae362832,434 - Praça Procópio Ferreira II,-23.599550,-46.690740,2025-08-17T20:00:25.506774+00:00Z,3,12,592,1,1,...,True,0,3,"[key, transitcard, phone]",False,0.00,15,{},False,4.842361
4,0b509d7429586cacb0776fd8b203d61c,443 - Cardoso Melo III,-23.596440,-46.688550,2025-08-17T20:00:25.506569+00:00Z,0,15,556,1,1,...,True,0,0,"[key, transitcard, phone]",False,0.00,15,{},False,4.464039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,fd5d8d6dcc34499ffa8ca4db591e8e45,317 - R. Leonardo Nunes,-23.598733,-46.639103,2025-08-17T20:00:25.505845+00:00Z,1,14,317,1,1,...,True,0,1,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False,6.370210
236,fdb890213c54ad952730bd9fab78a147,76 - 8° Batalhão,-23.574298,-46.654040,2025-08-17T20:00:25.504419+00:00Z,2,13,76,1,1,...,True,0,2,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False,3.429161
237,fe23e24943b116bf2b7bbfe856292ffa,131 - Ibirapuera Portão 06,-23.593685,-46.660042,2025-08-17T20:00:25.504800+00:00Z,13,6,131,1,1,...,True,0,13,"[key, transitcard, phone]",False,NaN,19,{},False,4.657215
238,ff724047677a832632b2ec87805270c0,215 - E.E. Godofredo Furtado,-23.559442,-46.678456,2025-08-17T20:00:25.505348+00:00Z,1,14,215,1,1,...,True,0,1,"[key, transitcard, creditcard, phone]",True,NaN,15,{},False,0.448757


In [125]:
for test in df_stations:
    print(df_stations.loc[test])

KeyError: 'id'

In [93]:
from geopy.distance import geodesic

p1 = (-23.5505, -46.6333)
p2 = (-22.9068, -43.1729)
print(geodesic(p1, p2).km)

361.2608611455548
